<hr>

# Hubzu

In [ ]:
import pandas as pd
df = pd.read_csv('Hubzu.csv')

df = df.dropna(subset=['Address', 'Date']) #drop null values from Address & Date

#Splits date to date & time
df.Date = df.Date.str.replace('at', ',')
df[['Date', 'Time']] = df['Date'].str.split(',', n=1, expand=True)
df.Time = df.Time.str.replace('local', '')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])

#Keeps valid date range
start_date = pd.to_datetime('2025-07-02')
end_date = pd.to_datetime('2025-08-02')
df = df[df['Date'] >= start_date]
df = df[df['Date'] <= end_date]
df = df.sort_values(by='Date')

#Splits address2 to city, state * postal
df[['City', 'Text2']] = df['Address2'].str.split(',', n=1, expand=True)
df[['Blank', 'State', 'Postal']] = df['Text2'].str.split('  ', n=2, expand=True)
df = df.drop(['Address2','Text2' , 'Blank'], axis=1)

#Drops address in case column
keywords = ['ST', 'WY', 'DR']
pattern = '|'.join(keywords)
df.Case = df.Case.str.upper()
df['Case'] = df['Case'].where(~df['Case'].isin(df['Address']), '')
df['Case'] = df['Case'].where(~df['Case'].str.contains(pattern, case=False, na=False), '')

df = df.drop_duplicates(subset=['Address']) #Removes duplicate address

#Create the clean csv with correct column order
cols = ['Address', 'City', 'State', 'Postal', 'County', 'Date', 'Time', 'Case', 'URL', 'Source', 'Atty', 'Atty_Contact']
df = df.reindex(columns=cols)
df.to_csv('Hubzu2.csv', index=False)

<hr>

# Xome

In [ ]:
import pandas as pd
df = pd.read_csv('Xome.csv')

#Creates the county column
county_cols = ['Text', 'Text1', 'Text2', 'Text3', 'Text4', 'Text5', 'Text6', 'Text7', 'Text8', 'Text9']
df['County'] = df.apply(lambda row: next((str(row[col]) for col in county_cols if 'County' in str(row[col])), None), axis=1)
df = df.drop(['Text', 'Text1', 'Text2', 'Text3', 'Text4', 'Text5', 'Text6', 'Text7', 'Text8', 'Text9'], axis=1)
df['County'] = df['County'].str.replace(r'\s+', '', regex=True) #removes newlines and whitespaces
df['County'] = df['County'].str.replace('MunicipalityCounty:', '')

#Splits address2 to city, state and postal
df['City'] = df['Address2'].str.rsplit(" ", n=2).str[0]
df['State'] = df['Address2'].str.split(" ").str[-2]
df['Postal'] = df['Address2'].str.split(" ").str[-1]
df = df.drop(['Address2'], axis=1)

#Splits date into date & time
df = df.dropna(subset='Date')
df['Date'] = df['Date'].str.replace(r'\s+', '', regex=True) #removes newlines and whitespaces
df[['Date', 'Extra']] = df['Date'].str.split('-', expand=True)
df = df.drop(['Extra'], axis=1)
df[['Date', 'Time']] = df['Date'].str.split(',', expand=True)

#Parses date into correct format and filters within date range
start_date = pd.to_datetime('2025-07-02')
end_date = pd.to_datetime('2025-08-02')
df['date_fixed'] = df['Date'].str.replace(r'([a-zA-Z]+)(\d+)', r'\1 \2', regex=True)
df['date_with_year'] = df['date_fixed'] + ' 2025'
df['Date'] = pd.to_datetime(df['date_with_year'], format='%B %d %Y', errors='coerce')
df = df[df['Date'] >= start_date]
df = df[df['Date'] <= end_date]
df = df.sort_values(by='Date')
df = df.drop(['date_fixed', 'date_with_year'], axis=1)

df = df.drop_duplicates(subset=['Address']) #Removes duplicate address

#Creates clean csv with proper column order
cols = ['Address', 'City', 'State', 'Postal', 'County', 'Date', 'Time', 'Case', 'URL', 'Source', 'Atty', 'Atty_Contact']
df = df.reindex(columns=cols)
df.to_csv('Xome2.csv', index=False)

<hr>

# Realforeclose
### Pre-clean the Details column in Excel prior to using pandas.
- Remove blank dates.
- Clean judgment and parcel columns.
- Remove row without address.
- Clean address2 column.
- Clean value column.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('RFC.csv')

#Splits date into date & time
df['Date'] = df['Date'].str.replace('\n', '')
df['Date'] = df['Date'].str.replace('Auction Starts', '')
df[['Date', 'Time']] = df['Date'].str.split(' ', n=1, expand=True)

#Filters judgment value above 20K
df['Judgment'] = df['Judgment'].str.replace('$', '')
df['Judgment'] = df['Judgment'].str.replace(',', '')
df['Judgment'] = pd.to_numeric(df['Judgment'], errors='coerce')
df = df[(df['Judgment'].isna()) | (df['Judgment'] >= 20000)]

#Removes rows with Timeshare and retain rows with Address
#Splits Address2 into city, state and postal
df = df[~df['Parcel'].str.contains('TIMESHARE', case=False, na=False)]
df[['City', 'Postal']] = df['Address2'].str.split(',', expand=True)
df['Postal'] = df['Postal'].str.replace('NO ZIP', '')

#Filters values >= 100K but <= 700K
df['Value'] = df['Value'].str.replace('$', '')
df['Value'] = df['Value'].str.replace(',', '')
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df[(df['Value'].isna()) | (df['Value'] >= 100000)]
df = df[(df['Value'].isna()) | (df['Value'] <= 700000)]

#Identifies the states
df.loc[df['Postal'].str.contains('CO', case=False, na=False), 'State'] = 'CO'
df.loc[df['Postal'].str.contains('FL', case=False, na=False), 'State'] = 'FL'
df.loc[df['URL'].str.contains('ohio', case=False, na=False), 'State'] = 'OH'
df.loc[df['URL'].str.contains('pa.', case=False, na=False), 'State'] = 'PA'
df['State'] = df['State'].fillna('FL')
df['County'] = df['URL'].str.extract(r'//([^\.]+)')
df['Postal'] = df['Postal'].str.replace('CO-', '')
df['Postal'] = df['Postal'].str.replace('FL-', '')

df = df.drop_duplicates(subset=['Address']) #Removes duplicate address

#Creates clean csv with proper column order
cols = ['Address', 'City', 'State', 'Postal', 'County', 'Date', 'Time', 'Case', 'URL', 'Source', 'Atty', 'Atty_Contact']
df = df.reindex(columns=cols)
df.to_csv('RFC2.csv', index=False)

<hr>

# Auction
- Pre-clean the Atty column.
- Post-clean the city, state, and county columns.

In [ ]:
import pandas as pd

df = pd.read_csv('/home/lostinthecode/RegMichel/input/Auction.csv', encoding="latin-1")

# Creates city, state, and postal columns
df['Address3'] = df['Address2'].str.replace(r',,+', ',', regex=True)
df['City'] = df['Address3'].str.split(',').str[-3]
df['StateZip'] = df['Address3'].str.split(',').str[-2].str.strip()
df[['State', 'Postal']] = df['StateZip'].str.split(' ', expand=True)
df['County'] = df['Address3'].str.split(',').str[-1]
df['City'] = df['City'].str.replace(r'\s*\(.*', '', regex=True)

# Creates atty_contact column
df['Atty_Contact'] = df['Atty_Contact'].str.split('call', n=1).str[1].str.strip()

# Removes unnecessary keywords from address
df['Address'] = df['Address'].str.replace(' aka ', ',', case=False)
df['Address'] = df['Address'].str.replace(' a/k/a ', ',', case=False)
df['Address'] = df['Address'].str.replace(' fka ', ',', case=False)
df['Address'] = df['Address'].str.replace(' f/k/a ', ',', case=False)
df['Address'] = df['Address'].str.replace(' nka ', ',', case=False)
df['Address'] = df['Address'].str.replace(' n/k/a ', ',', case=False)
df['Address'] = df['Address'].str.replace(' arta ', ',', case=False)
df[['Address', 'Extra2']] = df['Address'].str.split(',', n=1, expand=True)

# Filters value >= 100K and <= 700K
df['Value'] = df['Value'].str.replace('$', '')
df['Value'] = df['Value'].str.replace(',', '')
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
df = df[(df['Value'].isna()) | (df['Value'] >= 100000)]
df = df[(df['Value'].isna()) | (df['Value'] <= 700000)]

# Transforms date and time into their proper format
df['Date'] = df['Date'].str.split('\n').str[0]
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date'] = df['Date'].fillna(df['Date2'])

df['Time2'] = df['Time2'].str.split('-').str[0]
df['Time'] = df['Time'].str.replace('TBD', '', regex=True)
df['Time'] = df['Time'].fillna(df['Time2'])

# # Keeps valid date range
df = df.dropna(subset='Date')
start_date = pd.to_datetime('2025-07-02')
end_date = pd.to_datetime('2025-08-02')
df = df[df['Date'] >= start_date]
df = df[df['Date'] <= end_date]

# Creates clean csv with proper column order
cols = ['Address', 'City', 'State', 'Postal', 'County', 'Date', 'Time', 'Case', 'URL', 'Source', 'Atty', 'Atty_Contact']
df = df.reindex(columns=cols)
df.to_csv('Auction2.csv', index=False)

<hr>

# Masterlist

In [ ]:
import pandas as pd
files = ['RFC2.csv', 'Xome2.csv', 'Auction2.csv', 'Hubzu2.csv']
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

#Convert to title case
df.Address = df.Address.str.title()
df.City = df.City.str.title()
df.County = df.County.str.title()
df.Atty = df.Atty.str.title()

#Limit postal code to first 5 digits
df['Postal'] = df['Postal'].astype(str)
df.Postal = df.Postal.str[:5]

#Replace/remove symbols
df.Address = df.Address.str.replace('.', '')
df.Address = df.Address.str.replace(',', '')
df.Address = df.Address.str.replace('#', '')
df.Address = df.Address.str.replace('& ', '')
df.Address = df.Address.str.replace('-', ' ')

#Abbreviate
df.Address = df.Address.str.replace('Street', 'St')
df.Address = df.Address.str.replace('Drive', 'Dr')
df.Address = df.Address.str.replace('Road', 'Rd')
df.Address = df.Address.str.replace('Lane', 'Ln')
df.Address = df.Address.str.replace('Avenue', 'Ave')
df.Address = df.Address.str.replace('Terrace', 'Ter')
df.Address = df.Address.str.replace('Circle', 'Cir')
df.Address = df.Address.str.replace('Court', 'Ct')
df.Address = df.Address.str.replace('Place', 'Pl')
df.Address = df.Address.str.replace('Boulevard', 'Blvd')
df.Address = df.Address.str.replace('Parkway', 'Pkwy')
df.Address = df.Address.str.replace('Ridge', 'Rdg')
df.Address = df.Address.str.replace('Trail', 'Trl')
df.Address = df.Address.str.replace('North', 'N')
df.Address = df.Address.str.replace('Northeast', 'NE')
df.Address = df.Address.str.replace('Northwest', 'NW')
df.Address = df.Address.str.replace('South', 'S')
df.Address = df.Address.str.replace('Southeast', 'SE')
df.Address = df.Address.str.replace('Southwest', 'SW')
df.Address = df.Address.str.replace('East', 'E')
df.Address = df.Address.str.replace('West', 'W')

#Capitalize common terms in Atty column
df.Atty = df.Atty.str.replace('Llc', 'LLC')
df.Atty = df.Atty.str.replace('llc', 'LLC')
df.Atty = df.Atty.str.replace('Llp', 'LLP')
df.Atty = df.Atty.str.replace('Pc', 'PC')

#Clean County column
df.County = df.County.str.replace('Myorangeclerk', 'My Orange Clerk')
df.County = df.County.str.replace('County', '')

#Removes duplicates in Address but fill null values with values of duplicate rows prior to dropping
df = df.groupby('Address', as_index=False).first()

# #Create new clean csv
df.to_csv('Clean_062825.csv', index=False)


<hr>

# DealMachine - Inactive, this is for update.

In [ ]:
import pandas as pd
import numpy as np 

df = pd.read_csv("DM_081824.csv")


df = df.drop(['contact_id', 'lead_id', 'associated_property_id', 'associated_property_address_full', 
              'associated_property_address_line_2', 'associated_parcel_id', 'primary_mailing_address',
              'primary_mailing_city', 'primary_mailing_state', 'primary_mailing_zip',
              'contact_flags', 'mailing_address_previous', 'mailing_address_zip_previous',
              'mailing_address_city_previous', 'mailing_address_state_previous', 'gender',
              'mob', 'language_preference', 'marital_status', 'est_household_income_code', 'home_business', 'net_asset_value',
              'education_model', 'occupation_group', 'occupation_code', 'business_owner',
              'phone_1_activity_status', 'phone_1_carrier', 'phone_1_prepaid_indicator',
              'phone_1_usage_2_months', 'phone_1_usage_12_months', 'phone_2_activity_status', 
              'phone_2_carrier', 'phone_2_prepaid_indicator', 'phone_2_usage_2_months', 
              'phone_2_usage_12_months', 'phone_3_activity_status_code', 'phone_3_owner',
              'phone_3_prepaid_indicator', 'phone_3_usage_2_months', 'phone_3_usage_12_months'], axis=1)

df.rename(columns={'associated_property_address_line_1': 'Address', 'associated_property_address_city': 'City',
                   'associated_property_address_state': 'State', 'associated_property_address_zipcode': 'Postal Code',
                   'first_name': 'First Name', 'last_name': 'Last Name', 'middle_initial': 'Middle Name',
                   'generational_suffix': 'Suffix Name', 'email_address_1': 'Email',
                   'email_address_2': 'Email - 2', 'email_address_3': 'Email - 3', 
                   'phone_1': 'Phone', 'phone_1_type': 'Phone Type', 'phone_2': 'Phone No. 2',
                   'phone_2_type': 'Phone Type (2)', 'phone_3': 'Phone No. 3', 
                   'phone_3_type': 'Phone Type (3)'}, inplace=True)

print(df.columns)

df.Email = df.Email.str.lower()
df['Email - 2'] = df['Email - 2'].str.lower()
df['Email - 3'] = df['Email - 3'].str.lower()

df['Phone Type'] = df['Phone Type'].str.replace('Wireless', 'Mobile')
df['Phone Type (2)'] = df['Phone Type (2)'].str.replace('Wireless', 'Mobile')
df['Phone Type (3)'] = df['Phone Type (3)'].str.replace('Wireless', 'Mobile')

df['Phone'] = df['Phone'].replace({'DNC Excluded': '', 'DNC Excluded, Landline Excluded': '', ', Landline Excluded': '', 'Landline Excluded': ''}, regex=True).replace('', np.nan)
df['Phone No. 2'] = df['Phone No. 2'].replace({'DNC Excluded': '', 'DNC Excluded, Landline Excluded': '', ', Landline Excluded': '', 'Landline Excluded': ''}, regex=True).replace('', np.nan)
df['Phone No. 3'] = df['Phone No. 3'].replace({'DNC Excluded': '', 'DNC Excluded, Landline Excluded': '', ', Landline Excluded': '', 'Landline Excluded': ''}, regex=True).replace('', np.nan)

df['Phone'] = df['Phone'].combine_first(df['Phone No. 2'])
df['Phone Type'] = df['Phone Type'].combine_first(df['Phone Type (2)'])

df['Phone No. 2'] = df.apply(lambda row: row['Phone No. 2'].replace(float(row['Phone']), ''), axis=1)

# df['Phone No. 2'] = df['Phone No. 2'].combine_first(df['Phone No. 3'])
# df['Phone Type (2)'] = df['Phone Type (2)'].combine_first(df['Phone Type (3)'])

df['Phone No. 2'] = df['Phone No. 2'].where(~df['Phone'])
df = df.sort_values('Phone', ascending=False, na_position='last')

df.to_csv('clean_DM_081824.csv')
df.head()